In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
# Modify your starting parameters as necessary

months_in_a_year = 12
start_year = 2025
number_quarters = 13

In [ ]:
class RevenueStream:
    """
    A class representing a revenue stream.
    """
    def __init__(self, name,
                 revenue_yearly,
                 cogs_yearly,
                 customer_acquisition_cost,
                 retention_rate_yearly,
                 n_customers_yearly,
                 fixed_costs_yearly,
                 max_n_customers=1000000,
                 is_subscription=True,
                 investment=0):
        self.name = name # Name of the revenue stream
        self.revenue_yearly = revenue_yearly  # Yearly subscription revenue, or total revenue for a one-off contract if not a subscription
        self.cogs_yearly = cogs_yearly  # Cost of Goods Sold. For yearly subscription, or for a one-off contract if not a subscription
        self.customer_acquisition_cost = customer_acquisition_cost  # Customer Aquisition Cost. Occurs only once per subscribing customer or one-off contract.
        self.n_customers_yearly = n_customers_yearly # Number of customers added yearly
        self.retention_rate_yearly = retention_rate_yearly  # Customer retention rate. One-off contracts should have a 0% retention rate
        self.fixed_costs_yearly = fixed_costs_yearly # Fixed costs related to that revenue stream, e.g. servers for SaaS
        self.max_n_customers = max_n_customers # Maximum number of customers
        self.is_subscription = is_subscription # Whether the revenue stream is a subscription or one-off (e.g. consultancy revenue)
        self.investment = investment # Initial investment related to that revenue stream


class RevenueStreamQuarter:
    """A class representing quarterly data for a revenue stream."""
    def __init__(self):
        self.n_customers = 0
        self.cumulative_revenue = 0
        self.cumulative_costs = 0

    def __repr__(self):
        return vars(self)

In [ ]:
# Modify these assumptions and revenue stream names to match your use case

# Example consultancy revenue stream. No subscription. Low margin.
consultancy_revenue_stream = RevenueStream("Consultancy", 
                                      revenue_yearly=50_000, 
                                      cogs_yearly=30_000, 
                                      customer_acquisition_cost=4_000, 
                                      retention_rate_yearly=0, 
                                      n_customers_yearly=8, 
                                      fixed_costs_yearly=0, 
                                      max_n_customers=20,
                                      is_subscription=False)

# Example Software as a Service revenue stream. Subscription based. High margin. High number of customers.
saas_revenue_stream = RevenueStream("SaaS",
                                    revenue_yearly=480,
                                    cogs_yearly=48,
                                    customer_acquisition_cost=240,
                                    retention_rate_yearly=0.95,
                                    n_customers_yearly=1000,
                                    fixed_costs_yearly=10_000,
                                    max_n_customers=300000,
                                    is_subscription=True,
                                    investment=200_000)

# Example premium Software as a Service revenue stream. Subscription based. High margin. Low number of customers.
saaspremium_revenue_stream = RevenueStream("SaaS Premium",
                                       revenue_yearly=4_800,
                                       cogs_yearly=480,
                                       customer_acquisition_cost=2_400,
                                       retention_rate_yearly=0.95,
                                       n_customers_yearly=50,
                                       fixed_costs_yearly=20_000,
                                       max_n_customers=100,
                                       is_subscription=True,
                                       investment=200_000)

In [ ]:
def calculate_quarterly_data(revenue_stream, n_quarters=12):
    quarters = []
    quarterly_customers_growth = revenue_stream.n_customers_yearly / 4
    quarterly_fixed_costs = revenue_stream.fixed_costs_yearly / 4
    if revenue_stream.is_subscription:
        quarterly_revenue = revenue_stream.revenue_yearly / 4
        quarterly_cogs = revenue_stream.cogs_yearly / 4
        retention_rate_quarterly = revenue_stream.retention_rate_yearly ** (1 / 4)
    else:
        quarterly_revenue = revenue_stream.revenue_yearly
        quarterly_cogs = revenue_stream.cogs_yearly
        retention_rate_quarterly = 0

    for _ in range(n_quarters):
        quarter_data = RevenueStreamQuarter()

        # Get the previous quarter's customers or start with 0
        previous_customers = quarters[-1].n_customers if quarters else 0

        # Recurring services: Apply churn (loss of customers) and add new growth
        # For one-off services, retention_rate_quarterly will be zero. Only count the number of new customers for that quarter.
        n_customers = previous_customers * retention_rate_quarterly + quarterly_customers_growth
        n_customers = round(min(n_customers, revenue_stream.max_n_customers), 1)

        # Calculate cumulative revenue and costs based on current customers
        quarter_data.n_customers = n_customers
        cumulative_revenue_previous_quarter = quarters[-1].cumulative_revenue if quarters else 0
        quarter_data.cumulative_revenue = int(cumulative_revenue_previous_quarter + (n_customers * quarterly_revenue))
        cumulative_costs_previous_quarter = quarters[-1].cumulative_costs if quarters else revenue_stream.investment
        quarter_data.cumulative_costs = int(cumulative_costs_previous_quarter + (n_customers * quarterly_cogs) + quarterly_fixed_costs)

        quarters.append(quarter_data)

    return quarters

In [ ]:
def generate_quarter_labels(start_year, n_quarters):
    quarter_labels = []
    year = start_year
    for i in range(n_quarters):
        quarter = (i % 4) + 1
        quarter_labels.append(f"{year} Q{quarter}")
        if quarter == 4:
            year += 1
    return quarter_labels

In [ ]:
quarter_labels = generate_quarter_labels(start_year, 12)

revenue_streams = [consultancy_revenue_stream, saas_revenue_stream, saaspremium_revenue_stream]

data = {}

# Calculate and add quarterly data for each revenue stream
for stream in revenue_streams:
    quarters = calculate_quarterly_data(stream)
    data[f'{stream.name} Revenue'] = [q.cumulative_revenue for q in quarters]
    data[f'{stream.name} Costs'] = [q.cumulative_costs for q in quarters]
    data[f'{stream.name} Customers'] = [q.n_customers for q in quarters]

quarterly_df = pd.DataFrame(data, index=quarter_labels)

quarterly_df

In [ ]:
# Calculate profit for each quarter, by subtracting costs from revenues
for revenue_stream in revenue_streams:
    revenue_stream_name = revenue_stream.name
    quarterly_df[f"{revenue_stream_name} Profit"] = quarterly_df[f"{revenue_stream_name} Revenue"] - quarterly_df[f"{revenue_stream_name} Costs"]

# Calculate total gross for each quarter, combining data from all revenue streams
for suffix in ["Revenue", "Costs", "Customers", "Profit"]:
    columns = [f'{stream.name} {suffix}' for stream in revenue_streams]
    quarterly_df[f"Gross {suffix}"] = quarterly_df[columns].sum(axis=1)

quarterly_df = quarterly_df.sort_index(axis=1)

quarterly_df

In [ ]:
# Plotting settings and helper functions

# Number of shades per type
n_shades = 4

# Predefined list of color maps
base_color_maps = ["Greens", "Blues", "Reds", "Purples", "Oranges"]

def generate_color_gradient(base_color, n_shades):
    """Generates a gradient of colors from a base color."""
    cmap = plt.get_cmap(base_color)
    return [rgb_to_hex(cmap(0.2 + 0.6 * i / (n_shades - 1))) for i in range(n_shades)]

def rgb_to_hex(rgb):
    """Convert an RGB tuple to a HEX string."""
    return '#' + ''.join(f'{int(x * 255):02X}' for x in rgb[:3])

In [ ]:
# Include Gross numbers (sum of all revenue streams) in plots
revenue_stream_names = [stream.name for stream in revenue_streams] + ["Gross"]

# Create base colors dynamically
plot_colors = {}
for n, revenue_stream_name in enumerate(revenue_stream_names):
    base_color = base_color_maps[n % len(base_color_maps)]  # Cycle through base colors
    plot_colors[revenue_stream_name] = list(reversed(generate_color_gradient(base_color, n_shades)))

# First plot for Revenue, Profit, and Costs for all revenue streams
fig1 = go.Figure()

for n, revenue_stream_name in enumerate(revenue_stream_names):
    for i, suffix in enumerate(["Revenue", "Profit", "Costs"]):  # Removed "Customers"
        color = plot_colors[revenue_stream_name][i]
        colname = f'{revenue_stream_name} {suffix}'

        fig1.add_trace(go.Scatter(
            x=quarterly_df.index,
            y=quarterly_df[colname],
            mode='lines+markers',
            name=colname,
            line=dict(color=color),
            hoverinfo='name+text'
        ))

fig1.update_layout(
    title="Revenue Overview",
    xaxis_title="Quarter",
    yaxis_title="Euros",
    legend=dict(
        orientation="v",  # Vertical orientation
        yanchor="middle",
        y=0.5,  # Centered vertically
        xanchor="left",
        x=1.05  # Shifted slightly to the right of the plot
    ),
    barmode='group'
)

fig1.show()

# Second plot for Number of Customers
fig2 = go.Figure()

for n, revenue_stream_name in enumerate(revenue_stream_names):
    suffix = "Customers"
    color = plot_colors[revenue_stream_name][0]  # Use one of the colors from the gradient
    colname = f'{revenue_stream_name} {suffix}'

    fig2.add_trace(go.Scatter(
        x=quarterly_df.index,
        y=quarterly_df[colname],
        mode='lines+markers',
        name=colname,
        line=dict(color=color)
    ))

fig2.update_layout(
    title="Customers Overview",
    xaxis_title="Quarter",
    yaxis_title="Customers",
    legend=dict(
        orientation="v",  # Vertical orientation
        yanchor="middle",
        y=0.5,  # Centered vertically
        xanchor="left",
        x=1.05  # Shifted slightly to the right of the plot
    )
)

fig2.show()